In [ ]:
focus = ["focus_i", "focus_you_sg", "focus_he", "focus_we", "focus_you_pl","focus_they"]
focus = [x+"_"+y for y in ["subj", "obj", "poss"]for x in focus]

Error: Session cannot generate requests